In [5]:
import os
import sys
import numpy as np
import pandas as pd
import os.path

data_path="/headnode1/abry4213/data/TS_feature_manuscript/"

%load_ext rpy2.ipython

In [ ]:
%%R
# Load tidyverse R package
suppressPackageStartupMessages({
    library(tidyverse)
    library(see)
    library(cowplot)
    theme_set(theme_cowplot())
})

In [12]:
if not os.path.isfile(f"{data_path}/time_series_features/ABIDE_pyspi14_mean_FC.feather"):
    # Load pyspi14 data for UCLA CNP and ABIDE
    UCLA_CNP_pyspi14 = pd.read_feather(f"{data_path}/time_series_features/UCLA_CNP_pyspi14_filtered.feather")
    ABIDE_pyspi14 = pd.read_feather(f"{data_path}/time_series_features/ABIDE_pyspi14_filtered.feather")

    # Take the mean value per SPI by Sample_ID
    UCLA_CNP_pyspi14_mean = UCLA_CNP_pyspi14.groupby(["Sample_ID", "SPI"])["value"].mean().reset_index()
    ABIDE_pyspi14_mean = ABIDE_pyspi14.groupby(["Sample_ID", "SPI"])["value"].mean().reset_index()

    # Save results to feather files
    UCLA_CNP_pyspi14_mean.to_feather(f"{data_path}/time_series_features/UCLA_CNP_pyspi14_mean_FC.feather")
    ABIDE_pyspi14_mean.to_feather(f"{data_path}/time_series_features/ABIDE_pyspi14_mean_FC.feather")
else:
    UCLA_CNP_pyspi14_mean = pd.read_feather(f"{data_path}/time_series_features/UCLA_CNP_pyspi14_mean_FC.feather")
    ABIDE_pyspi14_mean = pd.read_feather(f"{data_path}/time_series_features/ABIDE_pyspi14_mean_FC.feather")